In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))



In [2]:
import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return (int(article_id), words)
    except ValueError as e:
        return (undefined,undefined)

def filter_word(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return ('narodnaya' in words) or ('Narodnaya' in words)
    except ValueError as e:
        return False
    
wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).filter(filter_word).map(parse_article)
wiki.cache()
result = wiki.take(6)

In [3]:
def numeroter_mot(parsed_article):
    try:
        article_id = parsed_article[0]
        words = parsed_article[1]
        ret_val = []
        for idx, val in enumerate(words):
            ret_val.append( (str(article_id) + "_" + str(idx), val.lower()))
    except ValueError as e:
        return []
    return ret_val

def numeroter_mot_precedent(parsed_article):
    try:
        article_id = parsed_article[0]
        words = parsed_article[1]
        ret_val = []
        for idx, val in enumerate(words):
            ret_val.append( (str(article_id) + "_" + str(idx + 1), val.lower()))
    except ValueError as e:
        return []
    return ret_val
                

In [4]:
#wiki = wiki.filter(lambda x: 'narodnaya' in x[1])
mots_numerotes = wiki.flatMap(numeroter_mot)
mots_numerotes.cache()
result = mots_numerotes.take(6)
mots_precedents_numerotes = wiki.flatMap(numeroter_mot_precedent)
mots_precedents_numerotes.cache()
result_precedents = mots_precedents_numerotes.take(6)


In [5]:
mots_precedents_numerotes_narodnaya = mots_precedents_numerotes.filter (lambda x: x[1] == 'narodnaya')
mots_precedents_numerotes_narodnaya.cache()


PythonRDD[13] at RDD at PythonRDD.scala:48

In [6]:
joined = mots_precedents_numerotes_narodnaya.join(mots_numerotes)
result_joined = joined.take(20)

In [7]:
def prepare_bigram(pair):
    return (pair[1][0] + "_" + pair[1][1],1)

def reduce_bigrams(a,b):
    return a+b

bigrams = joined.map(prepare_bigram).reduceByKey(reduce_bigrams)

In [8]:
results = bigrams.collect()

for result in results:
    print(("%s\t%s" % (result[0],result[1])).decode('utf-8'))


narodnaya_gazeta	1
narodnaya_volya	9
